# 3. Search

## 3.2 A\* and Related Searchers
To solve a search problem, we can construct a `Searcher` object for the problem and then repeatedly ask for the next path using search. If there are no more paths, `None` is returned.

In [ ]:
from aipython.searchProblem import Path
from aipython.searchGeneric import Frontier
from aispace2.jupyter.search import Displayable, visualize


class Searcher(Displayable):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    This does depth-first search unless overridden
    """

    def __init__(self, problem):
        """creates a searcher from a problem
        """
        self.problem = problem
        self.initialize_frontier()
        self.num_expanded = 0
        self.add_to_frontier(Path(problem.start_node()))
        # self.display(3,"Frontier:",self.frontier)
        super().__init__()

    def initialize_frontier(self):
        self.frontier = []

    def empty_frontier(self):
        return self.frontier == []

    def add_to_frontier(self, path):
        self.frontier.append(path)

    @visualize
    def search(self):
        """returns (next) path from the problem's start node
        to a goal node. 
        Returns None if no path exists.
        """
        while not self.empty_frontier():
            path = self.frontier.pop()
            self.display(2, "Expanding: ", path, "(cost:", path.cost, ")")
            self.num_expanded += 1
            if self.problem.is_goal(path.end()):    # solution found
                self.display(1, self.num_expanded, "paths have been expanded and",
                             len(self.frontier), "paths remain in the frontier")
                self.solution = path   # store the solution found
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3, "Neighbors are", neighs)
                for arc in neighs:
                    self.add_to_frontier(Path(path, arc))
                self.display(3, "Frontier:", self.frontier)
        self.display(1, "No (more) solutions. Total of",
                     self.num_expanded, "paths expanded.")


class AStarSearcher(Searcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """

    def __init__(self, problem):
        super().__init__(problem)

    def initialize_frontier(self):
        self.frontier = Frontier()

    def empty_frontier(self):
        return self.frontier.empty()

    def add_to_frontier(self, path):
        """add path to the frontier with the appropriate cost"""
        value = path.cost + self.problem.heuristic(path.end())
        self.frontier.add(path, value)

### A\* Visualization
We first construct an instance of `AStarSearcher` and display it. Then, in the next cell, we call `search()` on the `Searcher` in order to request the next path. Once we find a solution, we can call `search()` again to find other solutions, if any.

In [ ]:
from aipython.searchProblem import problem1, problem2, acyclic_delivery_problem
from IPython.display import display

s = AStarSearcher(acyclic_delivery_problem)
s.sleep_time = 0.2
s.show_edge_costs = True
s.show_node_heuristics = True
display(s)

In [ ]:
s.search()

### A\* Search with Multiple Path Pruning

We can cut down on the number of paths we explore if we check to see if we have already visited before. By keeping a `explored` set, and only expanding the path if it isn't in `explored`, we can implement multiple path pruning. This is done below by overriding the `search` method.

Compare no pruning and multiple path pruning for the `cyclic_delivery_problem`. Which works better in terms of number of paths expanded, computational time or space?

In [ ]:
from aipython.searchProblem import Path
from aispace2.jupyter.search import Displayable, visualize


class SearcherMPP(AStarSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """

    def __init__(self, problem):
        super().__init__(problem)
        self.explored = set()

    @visualize
    def search(self):
        """returns next path from an element of problem's start nodes
        to a goal node. 
        Returns None if no path exists.
        """
        while not self.empty_frontier():
            path = self.frontier.pop()
            if path.end() not in self.explored:
                self.display(2, "Expanding: ", path, "(cost:", path.cost, ")")
                self.explored.add(path.end())
                self.num_expanded += 1
                if self.problem.is_goal(path.end()):
                    self.display(1, self.num_expanded, "paths have been expanded and",
                                 len(self.frontier), "paths remain in the frontier")
                    self.solution = path   # store the solution found
                    return path
                else:
                    neighs = self.problem.neighbors(path.end())
                    for arc in neighs:
                        self.add_to_frontier(Path(path, arc))
                    self.display(3, "Frontier:", self.frontier)
        self.display(1, "No (more) solutions. Total of",
                     self.num_expanded, "paths expanded.")

In [ ]:
from aipython.searchProblem import problem1, problem2, acyclic_delivery_problem
from IPython.display import display

s_mpp = SearcherMPP(problem2)
s_mpp.sleep_time = 0.2
display(s_mpp)

In [ ]:
s_mpp.search()

## 3.3 Branch-and-bound Search
This uses depth- first search to find a path to a goal that extends path with cost less than the bound. Once a path to a goal has been found, that path is remembered as the best path, the bound is reduced, and the search continues.

In [ ]:
from aipython.searchProblem import Path
from aispace2.jupyter.search import Displayable, visualize


class DF_branch_and_bound(Searcher):
    """returns a branch and bound searcher for a problem.    
    An optimal path with cost less than bound can be found by calling search()
    """

    def __init__(self, problem, bound=float("inf")):
        """creates a searcher than can be used with search() to find an optimal path.
        bound gives the initial bound. By default this is infinite - meaning there
        is no initial pruning due to depth bound
        """
        super().__init__(problem)
        self.best_path = None
        self.bound = bound

    @visualize
    def search(self):
        """returns an optimal solution to a problem with cost less than bound.
        returns None if there is no solution with cost less than bound."""
        self.frontier = [Path(self.problem.start_node())]
        self.num_expanded = 0
        while self.frontier:
            path = self.frontier.pop()
            if path.cost + self.problem.heuristic(path.end()) < self.bound:
                self.display(3, "Expanding: ", path, "cost:", path.cost)
                self.num_expanded += 1
                if self.problem.is_goal(path.end()):
                    self.best_path = path
                    self.bound = path.cost
                    self.display(2, "New best path:",
                                 path, " cost:", path.cost)
                for arc in reversed(list(self.problem.neighbors(path.end()))):
                    self.add_to_frontier(Path(path, arc))
        self.display(1, "Number of paths expanded:", self.num_expanded)
        self.solution = self.best_path
        return self.best_path

In [ ]:
from aipython.searchProblem import problem1, problem2, acyclic_delivery_problem
from IPython.display import display

s_dfbb = DF_branch_and_bound(problem2)
s_dfbb.sleep_time = 0.2
display(s_dfbb)

In [ ]:
s_dfbb.search()